# Projeto de Sinais

## Sinais e Sistemas Dinâmicos - Prof. Derzu Omaia

## Jansepetrus Brasileiro Pereira e Nathália de Vasconcelos Silva

Descrição do projeto: https://www.dropbox.com/s/wzibx7g7136um4z/Projeto.pdf?dl=0

### Importar todas as bibliotecas necessárias para o projeto

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import fft

### Percorrer a pasta do dataset IRMAS, abrir todos os arquivos .wav e aplicar a transformada de Fourier

In [20]:
instruments = os.listdir('IRMAS-TrainingData_red/')

file_instruments = []
data = []

for x in range (0, 10):
    file_instruments.append(os.listdir('IRMAS-TrainingData_red/' + instruments[x])) 

for x in range (0, 10):
    for y in range (0, 99):
        data.append(wavfile.read('IRMAS-TrainingData_red/' + instruments[x] + '/' + file_instruments[x][y]))
        #fft_output = fft(data)

        #%matplotlib inline
        #plt.plot(data, np.abs(fft_output))
        #plt.show()

# Convertendo a lista 'data' para um dataframe 'df_data' e deletando a coluna referente ao sampling rate
df_data = pd.DataFrame(data)
df_data = df_data.drop(axis = 1, columns = 0)
print(df_data)
df_data.describe()

                                                     1
0    [[-661, -691], [-760, -863], [-784, -974], [-7...
1    [[-491, 42], [-445, 9], [-407, 19], [-383, 122...
2    [[1721, 6998], [-677, 7034], [-2536, 6820], [-...
3    [[-315, 144], [-171, 104], [-19, 72], [128, 42...
4    [[89, 319], [-15, 227], [-126, 101], [-219, -3...
5    [[998, 3036], [560, 2577], [349, 2097], [289, ...
6    [[-842, -453], [-920, -346], [-887, -145], [-7...
7    [[177, 1661], [269, 1473], [335, 1275], [381, ...
8    [[-767, -1451], [-705, -1477], [-610, -1476], ...
9    [[2222, 3990], [1844, 3918], [1914, 4191], [22...
10   [[-102, -772], [-102, -739], [-115, -700], [-1...
11   [[-61, 1102], [-189, 1146], [-307, 1255], [-38...
12   [[657, 56], [669, 151], [636, 224], [587, 289]...
13   [[-4, 154], [38, 152], [46, 167], [22, 192], [...
14   [[-447, -445], [-557, -530], [-707, -623], [-8...
15   [[-3, -143], [26, -83], [74, -25], [131, 25], ...
16   [[1101, -789], [991, -805], [846, -831], [670,...
17   [[343

,1
count,990
unique,990
top,"[[-3791, -4045], [-4049, -4278], [-4286, -4516..."
freq,1


### Converter o Numpy array dos dados da transformada de Fourier em Pandas DataFrame

#df_fft = pd.DataFrame(fft_output)
df_fft = pd.DataFrame(data)
df_fft.describe()